## データの取得(実行のみ)

ポケモン図鑑から取得(https://zukan.pokemon.co.jp)

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import urllib
import urllib.request

def create_csv(filename):
  with open(filename, "w", encoding="utf-8") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["idx", "name", "omosa", "takasa", "bunrui", "tokusei_1", "tokusei_2", "type_1", "type_2", "text_1", "text_2", "spec_hp", "spec_kougeki", "spec_bougyo", "spec_tokukou", "spec_tokubou", "spec_subayasa"])

def scraping_save(num, csv_filename):    
  site = requests.get("https://zukan.pokemon.co.jp/detail/" + num)
  data = BeautifulSoup(site.text, "html.parser")

  title_part = data.find_all("script", {"type": "application/json"})
  item = title_part[0].contents[0].strip()

  #ポケモンの情報
  target_info = item.split('],"evolutionsType"')[0]
  print(target_info)
  idx = target_info.split('"no":"')[1].split('"')[0]
  name = target_info.split('"name":"')[1].split('"')[0]
  omosa = target_info.split('"omosa":"')[1].split('"')[0]
  takasa = target_info.split('"takasa":"')[1].split('"')[0]
  bunrui = target_info.split('"bunrui":"')[1].split('"')[0]
  tokusei_1 = target_info.split('"tokusei_1":')[1].split(',')[0]
  tokusei_2 = target_info.split('"tokusei_2":')[1].split(',')[0]
  type_1 = target_info.split('"type_1":')[1].split(',')[0]
  type_2 = target_info.split('"type_2":')[1].split(',')[0]
  text_1 = target_info.split('"text_1":"')[1].split('"')[0]
  text_2 = target_info.split('"text_2":"')[1].split('"')[0]
  spec_hp = target_info.split('"spec_hp":')[1].split(',')[0]
  spec_kougeki = target_info.split('"spec_kougeki":')[1].split(',')[0]
  spec_bougyo = target_info.split('"spec_bougyo":')[1].split(',')[0]
  spec_tokukou = target_info.split('"spec_tokukou":')[1].split(',')[0]
  spec_tokubou = target_info.split('"spec_tokubou":')[1].split(',')[0]
  spec_subayasa = target_info.split('"spec_subayasa":')[1].split(',')[0]
  fig_url = target_info.split('"image_m":')[1].split('"')[1]
  fig_url = fig_url.replace('\/', '/')

  #　save
  with open(csv_filename, "a", encoding="utf-8") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow([idx, name, omosa, takasa, bunrui, tokusei_1, tokusei_2, type_1, type_2, text_1, text_2, spec_hp, spec_kougeki, spec_bougyo, spec_tokukou, spec_tokubou, spec_subayasa])

  return fig_url

def save_image(fig_url, n_):    
  response = requests.get(fig_url)
  image = response.content
  file_name = ("images/%s.png"%n_)

  with open(file_name, "wb") as f:
    f.write(image)
  done = 1
  return done

csv_filename = "pokemon_info.csv"

#画像の保存先
save_path = "images"
if not os.path.exists("images"):
  os.mkdir("images")

#csv
if not os.path.exists(csv_filename):
  create_csv(csv_filename)
    
for n in range(1,10):    
  #scraping
  n_ = str("%03d"%n)
  fig_url = scraping_save(n_, csv_filename)
  done = save_image(fig_url, n_)
  
  if done == 1:
    try:
      for j in range(1,10):
        n_ = str("%03d-%d"%(n, j) )
        fig_url = scraping_save(n_, csv_filename)
        _ = save_image(fig_url, n_)
    except:
      pass

## 画像編(復習)
参考: https://pystyle.info/pytorch-list-of-transforms/

### 準備(実行のみ)

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image
from PIL import ImageFilter

import torch
import torchvision
from torchvision import transforms

In [ ]:
def loadImg(path):
  img = Image.open(path)
  img.load()
  bg = Image.new("RGB", img.size, (255, 255, 255))
  bg.paste(img, mask=img.split()[3])
  return bg

def display(images):
  plt.figure(figsize=(15,8))
  plt.subplots_adjust(wspace=0, hspace=0)
  for i, img in enumerate(images):
    plt.subplot(1, len(images), i+1)
    plt.tick_params(labelbottom=False,labelleft=False,bottom=False,left=False)
    plt.imshow(img)
  plt.show()

In [ ]:
img = loadImg('images/001.png')
img

### 1. (128\*128)にサイズ変換する

In [ ]:
transform = 

img = transform(img)
img

### 2. 色彩を変化させる

In [ ]:
transform = 

img_ = [transform(img) for _ in range(5)]
display(img_)

### 3. ランダムに左右反転させる

In [ ]:
transform = 

img_ = [transform(img) for _ in range(5)]
display(img_)

### 4. ランダムに20度回転、縦横移動、拡大縮小、白でパディングをさせる(アフィン変換)

In [ ]:
transform = 

img_ = [transform(img) for _ in range(5)]
display(img_)

## 自作データセット編
参考: https://pystyle.info/pytorch-how-to-create-custom-dataset-class/

### 準備(実行のみ)

In [ ]:
# 画像表示用の関数
def display_loader(loader):
  plt.figure(figsize=(40,8))
  plt.subplots_adjust(wspace=0, hspace=0)
  loc = 1
  for epoch in range(3):
    t = []
    for img, tp in loader:
      plt.subplot(3, 16, loc)
      plt.tick_params(labelbottom=False,labelleft=False,bottom=False,left=False)
      plt.imshow(np.transpose(img,(0,2,3,1)).squeeze())
      loc += 1
      t.append(tp)
    print(t)
  plt.show()

### Sample. データセットの作成

- torch.utils.data.Datasetを継承
- `__init__`関数、`__getitem__`関数、`__len__`関数が必要

In [ ]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, transform = None):
    self.transform = transform

    self.data = [1, 2, 3, 4, 5, 6]
    self.label = [0, 1, 0, 1, 0, 1]

  def __getitem__(self, idx):
    out_data = self.data[idx]
    out_label = self.label[idx]

    return out_data, out_label

  def __len__(self):
    return len(self.data)

### Sample. データローダーの作成

- torch.utils.data.DataLoaderを用いて作成したデータセットをロードする



In [ ]:
dataset = MyDataset()
loader = torch.utils.data.DataLoader(dataset, shuffle=False, batch_size=1)

for x, y in loader:
  print("data:", x, "label:", y)

### 5. 画像とタイプを出力するデータセットを作成する
- images/*.pngから画像を取得
 - 注意: PNG画像がRGBAになっているので以下を利用しRGBに変換
```python
img = Image.open(path)
img.load()
bg = Image.new("RGB", img.size, (255, 255, 255))
bg.paste(img, mask=img.split()[3])
```
- pokemon_info.csvからポケモンのタイプを取得
 - pokemon_info.csvには以下の情報が含まれる
   - idx: 図鑑番号
   - name: 名前
   - omosa: おもさ
   - takasa: 高さ
   - bunrui: 分類
   - tokusei_1: 特性1
   - tokusei_2: 特性2
   - type_1: タイプ1
   - type_2: タイプ2
   - text_1: 説明文1
   - text_2: 説明文2
   - spec_hp: HP
   - spec_kougeki: こうげき
   - spec_bougyo: ぼうぎょ
   - spec_tokukou: とくこう
   - spec_tokubou: とくぼう
   - spec_subayasa: すばやさ

#### 準備(実行のみ)

In [ ]:
# ポケモン図鑑の順番にパスを取得
img_path = []
for i in range(1, 10):
  img_path.append("images/%03d.png" % i)
  for j in range(1, 10):
    if os.path.exists("images/%03d-%d.png" %( i, j)):
      img_path.append("images/%03d-%d.png" % (i, j))

### 5-1. データセットを作成する
- 画像とポケモンの情報のパスを指定し読み込む
- transformを受け取り画像に適用する
- RGBA画像をRGBに変換する
- typeはtensor, float型に変換する

In [ ]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, img_paths, info_path, transform=None):


  def __getitem__(self, idx):
    

  def __len__(self):
    


### 5-2. データローダーを作成しデータを表示する
- 128にリサイズしたのちランダム左右反転、ランダムアフィン変換、色彩の変換、tensor変換を行うtransformを適用する

In [ ]:
transform = 

dataset = 
loader = 

display_loader(loader)

### 6. 好きなようにデータセット作成する(オプション)
- スクレイピングや手持ちのデータを使用して新たなデータセットを作成する
- データを出力する

## 言語編
参考: https://qiita.com/kenta1984/items/5ba254afa8a524381c43

### 準備(実行のみ)

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install  mecab-python3

In [ ]:
# ポケモンの情報から説明文1とタイプ1を取得
info = pd.read_csv("pokemon_info.csv")
sentences = []
text = info["text_1"].values
type = info["type_1"].values
for t in text:
  # 空白で分割し最後の引用の部分を削除
  t = ''.join(t.split("\u3000")[:-1])
  # Google Transliterate APIを利用してひらがな→漢字に変換(ひらがなのみだとMeCabが不安定なため)
  site = requests.get("http://www.google.com/transliterate?langpair=ja-Hira|ja&text="+t)
  sentence = []
  for s in site.text.split("],"):
    s = s.split(",")
    s = s[1].replace('["',  "")
    sentence.append(s.replace('"',  ""))
  sentences.append("".join(sentence))
np.savetxt("text_type.csv", np.stack([sentences, type], 1), delimiter=",", fmt="%s")

In [ ]:
import torchtext
from torchtext.vocab import FastText
from torchtext.legacy import data

import MeCab

## 7. MeCabで分かち書きするtokenize関数を定義する

In [ ]:
def tokenize(text):


## 8. Fieldを定義して、TabularDatasetを作成する
- TEXT Fieldとlabel Fieldを作成する
- TEXT Fieldにはtokenizeを指定する

In [ ]:
# Fieldオブジェクトの作成
TEXT = 
LABEL = 

# CSVファイルを読み込み、TabularDatasetオブジェクトの作成
dataset = 

## 9. textとlabelを表示する

## 10. FastTextを使用してvocabを作成する
少し時間がかかる

### 11. 単語ごとの出現回数を表示する

### 12. 「灼熱」の語彙のインデックスを表示する

### 13. インデックス番号100の語彙を表示する

### 14. 「エネルギー」の単語ベクトルを表示する

### 15. イテレーターを作成し、バッチの中身(TEXTとLABEL)を確認する

In [ ]:
iter = 

for batch in iter:
  print()

### 16. 好きなようデータセット作成する(オプション)
- スクレイピングや手持ちのテキストデータを使用して新たなデータセットを作成する
- データを出力する